# GOOGLE Stock Article extractions

In [1]:
import requests

# api_key = '23f5208b7e2940aeb8b72c67195565b2'
api_key = 'bd9bb272078b41ebb0ee0a905d4a9532'
date_from = '2023-12-11'
date_to = '2024-01-08'
# Expanded list of sources
sources = 'bloomberg,cnbc,reuters,financial-times,techcrunch,the-wall-street-journal,the-verge,business-insider,the-economist,wired,engadget,bbc-news,fortune,techradar'

# Broadening the search query
query = 'Google OR "Alphabet Inc." OR "Sundar Pichai" OR "Google Search" OR "Android" OR "Google Cloud" OR "YouTube" OR "Google Ads" OR "Google services"'

url = f'https://newsapi.org/v2/everything?q={query}&from={date_from}&to={date_to}&sources={sources}&apiKey={api_key}'
response = requests.get(url)
data = response.json()

# Check for 'articles' in response and extract headlines with publication dates
if 'articles' in data:
    news_data = [(article['title'], article['publishedAt'][:10]) for article in data['articles']]
    for headline, date in news_data:
        print(f"Date: {date}, Headline: {headline}")
else:
    print("Error or no articles found. Response:")
    print(data)


Date: 2023-12-15, Headline: Google will update Maps to prevent authorities from accessing location history data
Date: 2023-12-31, Headline: Google Fixes Nearly 100 Android Security Issues
Date: 2023-12-12, Headline: Google's Android strategy looks like it's unravelling
Date: 2023-12-17, Headline: Epic Games CEO fears Google may still be able to stifle Play Store competitors
Date: 2023-12-11, Headline: Google Messages APK suggests you may soon able to edit messages after they're sent
Date: 2023-12-12, Headline: Google’s App Store Monopoly Ruled Illegal as Jury Sides With Epic
Date: 2023-12-12, Headline: Google loses app store antitrust suit brought by Fortnight in a decision that could cost the tech giant billions
Date: 2024-01-02, Headline: This $500 dog bed for humans was one of the trendiest gifts of the year
Date: 2023-12-15, Headline: Google will no longer hold onto people's location data in Google Maps — meaning it can't turn that info over to the police
Date: 2023-12-19, Headline

# Sentiment Analysis on News Articles

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = AutoModelForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

# Extract just the headlines from news_data
headlines = [headline for headline, _ in news_data]

# Tokenize and encode headlines
max_length = 512  # You can adjust this based on your requirements
inputs = tokenizer(headlines, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Predict sentiment
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Pair each sentiment score with its corresponding date
sentiment_data = []
for (headline, date), prediction in zip(news_data, predictions):
    sentiment_data.append({
        'date': date,
        'headline': headline,
        'sentiment_positive': prediction[0].item(),
        'sentiment_neutral': prediction[1].item(),
        'sentiment_negative': prediction[2].item()
    })


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [3]:
sentiment_data

[{'date': '2023-12-15',
  'headline': 'Google will update Maps to prevent authorities from accessing location history data',
  'sentiment_positive': 0.9999830722808838,
  'sentiment_neutral': 8.806889127299655e-06,
  'sentiment_negative': 8.047699338931125e-06},
 {'date': '2023-12-31',
  'headline': 'Google Fixes Nearly 100 Android Security Issues',
  'sentiment_positive': 0.13971535861492157,
  'sentiment_neutral': 0.20315328240394592,
  'sentiment_negative': 0.6571313738822937},
 {'date': '2023-12-12',
  'headline': "Google's Android strategy looks like it's unravelling",
  'sentiment_positive': 0.9727575778961182,
  'sentiment_neutral': 0.026935411617159843,
  'sentiment_negative': 0.00030709418933838606},
 {'date': '2023-12-17',
  'headline': 'Epic Games CEO fears Google may still be able to stifle Play Store competitors',
  'sentiment_positive': 1.4250867934606504e-06,
  'sentiment_neutral': 3.229193907827721e-06,
  'sentiment_negative': 0.9999953508377075},
 {'date': '2023-12-11'

# Historical Stock Data and Sentiment Analysis Combined

In [4]:
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler

# Convert sentiment_data to a DataFrame
sentiment_df = pd.DataFrame(sentiment_data)

# Convert date to datetime and set as index
sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
sentiment_df.set_index('date', inplace=True)

# Aggregate sentiment scores by date
average_sentiment = sentiment_df.groupby('date').mean()

# Download stock data
ticker_symbol = 'GOOG'
stock_data = yf.download(ticker_symbol, start='2023-12-09', end='2024-01-08')

# Selecting the required columns
stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

# Convert stock_data index to datetime
stock_data.index = pd.to_datetime(stock_data.index)

# Combine stock data with sentiment data
combined_data = stock_data.join(average_sentiment)

# Interpolate missing values
combined_data = combined_data.interpolate(method='time')

# Scale price-related columns
price_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close']
scaler_prices = StandardScaler()
combined_data[price_cols] = scaler_prices.fit_transform(combined_data[price_cols])

# Scale Volume column independently
scaler_volume = StandardScaler()
combined_data['Volume'] = scaler_volume.fit_transform(combined_data[['Volume']])

print(combined_data)

<ipython-input-4-afa03e4a573e>:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  average_sentiment = sentiment_df.groupby('date').mean()


[*********************100%%**********************]  1 of 1 completed
                Open      High       Low     Close  Adj Close    Volume  \
Date                                                                      
2023-12-11 -1.306279 -1.433318 -1.221650 -1.138396  -1.138396  0.168622   
2023-12-12 -1.467134 -1.508860 -1.238392 -1.467295  -1.467295  0.368250   
2023-12-13 -1.094247 -1.436338 -1.204909 -1.364901  -1.364901  0.256067   
2023-12-14 -1.028441 -1.359285 -1.732280 -1.603820  -1.603820  0.659391   
2023-12-15 -1.569497 -1.421229 -1.294197 -1.405239  -1.405239  3.436430   
2023-12-18 -1.294582 -0.348545 -0.976100 -0.365789  -0.365789  0.283434   
2023-12-19 -0.083791 -0.230701  0.050737 -0.083430  -0.083430 -0.199910   
2023-12-20  0.597645  1.071022  0.597642  0.400611   0.400611  1.032364   
2023-12-21  0.726329  0.754353  0.894254  1.064617   1.064617 -0.445428   
2023-12-22  1.124077  1.122993  1.335679  1.350077   1.350077 -0.407711   
2023-12-26  1.372667  1.333000 

In [5]:
import pandas as pd
df = pd.DataFrame(combined_data)
df

,Open,High,Low,Close,Adj Close,Volume,sentiment_positive,sentiment_neutral,sentiment_negative
Date,,,,,,,,,
2023-12-11,-1.306279,-1.433318,-1.221650,-1.138396,-1.138396,0.168622,0.664799,0.000038,0.335163
2023-12-12,-1.467134,-1.508860,-1.238392,-1.467295,-1.467295,0.368250,0.555821,0.003603,0.440576
2023-12-13,-1.094247,-1.436338,-1.204909,-1.364901,-1.364901,0.256067,0.736931,0.125832,0.137238
2023-12-14,-1.028441,-1.359285,-1.732280,-1.603820,-1.603820,0.659391,0.833851,0.000029,0.166120
2023-12-15,-1.569497,-1.421229,-1.294197,-1.405239,-1.405239,3.436430,0.849880,0.000049,0.150072
2023-12-18,-1.294582,-0.348545,-0.976100,-0.365789,-0.365789,0.283434,0.999972,0.000007,0.000021
2023-12-19,-0.083791,-0.230701,0.050737,-0.083430,-0.083430,-0.199910,0.599963,0.200006,0.200031
2023-12-20,0.597645,1.071022,0.597642,0.400611,0.400611,1.032364,0.819516,0.066825,0.113659
2023-12-21,0.726329,0.754353,0.894254,1.064617,1.064617,-0.445428,0.999990,0.000002,0.000008


# Transformer Architecture

In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from tensorflow.keras.callbacks import EarlyStopping

selected_features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sentiment_positive', 'sentiment_neutral', 'sentiment_negative']
data = combined_data[selected_features]

train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.15)
train_data = data.iloc[:train_size]
val_data = data.iloc[train_size:train_size + val_size]
test_data = data.iloc[train_size + val_size:]

# Prepare features and target for model training
X_train = train_data.drop('Close', axis=1)
y_train = train_data['Close']
X_val = val_data.drop('Close', axis=1)
y_val = val_data['Close']
X_test = test_data.drop('Close', axis=1)
y_test = test_data['Close']

# Define and compile the neural network model
model = Sequential()
model.add(Dense(32, activation='LeakyReLU', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(16, activation='LeakyReLU'))
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    X_train, y_train,
    epochs=50,  # Set back to 50 or an arbitrarily large number
    batch_size=8,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]  # Add the early stopping callback here
)

train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

# Convert the epochs range object to a list
epochs_list = list(epochs)

# Plotting the training and validation loss
loss_fig = go.Figure()
loss_fig.add_trace(go.Scatter(x=epochs_list, y=train_loss, mode='lines', name='Training Loss'))
loss_fig.add_trace(go.Scatter(x=epochs_list, y=val_loss, mode='lines', name='Validation Loss'))
loss_fig.update_layout(title='Training and Validation Loss per Epoch',
                       xaxis_title='Epochs',
                       yaxis_title='Loss',
                       legend_title='Type')
loss_fig.show()

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f'MSE: {mse}, MAE: {mae}, R^2: {r2}')

# Reshape y_test and predictions for inverse transform
y_test_reshaped = y_test.values.reshape(-1, 1)
predictions_reshaped = predictions.reshape(-1, 1)

num_scaled_cols = 5

# Create separate dummy arrays for inverse scaling
dummy_array_y_test = np.zeros((len(y_test_reshaped), num_scaled_cols))
dummy_array_predictions = np.zeros((len(predictions_reshaped), num_scaled_cols))

# Fill in the 'Close' column values in the dummy arrays
# Assuming 'Close' is the last of the scaled columns
dummy_array_y_test[:, -1] = y_test_reshaped.flatten()
dummy_array_predictions[:, -1] = predictions_reshaped.flatten()

# Inverse transform the 'Close' prices using the dummy arrays
y_test_original = scaler_prices.inverse_transform(dummy_array_y_test)[:, -1]
predictions_original = scaler_prices.inverse_transform(dummy_array_predictions)[:, -1]

# Extracting testing dates
test_dates = test_data.index

# Plotting with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_dates, y=y_test_original, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=test_dates, y=predictions_original, mode='lines', name='Predicted'))
fig.update_layout(title='Actual vs Predicted Stock Prices', xaxis_title='Date', yaxis_title='Stock Price', legend_title='Legend')
fig.show()

Epoch 1/50
2/2 [==============================] - 1s 181ms/step - loss: 2.6387 - val_loss: 0.1397
Epoch 2/50
2/2 [==============================] - 0s 35ms/step - loss: 0.4435 - val_loss: 0.0607
Epoch 3/50
2/2 [==============================] - 0s 33ms/step - loss: 0.2954 - val_loss: 0.2400
Epoch 4/50
2/2 [==============================] - 0s 31ms/step - loss: 0.3090 - val_loss: 0.1515
Epoch 5/50
2/2 [==============================] - 0s 37ms/step - loss: 0.3833 - val_loss: 0.0249
Epoch 6/50
2/2 [==============================] - 0s 33ms/step - loss: 0.1611 - val_loss: 5.4038e-04
Epoch 7/50
2/2 [==============================] - 0s 61ms/step - loss: 0.0655 - val_loss: 0.0208
Epoch 8/50
2/2 [==============================] - 0s 32ms/step - loss: 0.1657 - val_loss: 0.0389
Epoch 9/50
2/2 [==============================] - 0s 31ms/step - loss: 0.2338 - val_loss: 0.0359
Epoch 10/50
2/2 [==============================] - 0s 32ms/step - loss: 0.1179 - val_loss: 0.0167
Epoch 11/50
2/2 [=======

1/1 [==============================] - 0s 88ms/step
MSE: 0.017393820213018975, MAE: 0.1207706609934849, R^2: 0.8702871522293966


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 833 (3.25 KB)
Trainable params: 833 (3.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
